## Insert markdown comments

In [1]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe

import os

tf.enable_eager_execution()

In [2]:
def parse_csv(line):
    example_defaults = [[0.], [0.], [0.], [0.], [0]]
    parsed_line = tf.decode_csv(line, example_defaults)
    features = tf.reshape(parsed_line[:-1], shape = ((4,)))
    labels = tf.reshape(parsed_line[-1], shape = (()))
    return features, labels   

def loss(model, x, y):
    y_ = model(x)
    return tf.losses.sparse_softmax_cross_entropy(labels = y, logits = y_)

def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets)
    return tape.gradient(loss_value, model.variables) # returns the gradient of the loss function with respect to all the model variables

In [3]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation = "relu", input_shape = (4,)),
    tf.keras.layers.Dense(10, activation = "relu"),
    tf.keras.layers.Dense(3)
])

In [5]:
train_dataset_url = "http://download.tensorflow.org/data/iris_training.csv"
train_dataset_fp = tf.keras.utils.get_file(fname = os.path.basename(train_dataset_url), origin=train_dataset_url)

train_dataset = tf.data.TextLineDataset(train_dataset_fp)

train_dataset = train_dataset.skip(1)             # skip the first header row
train_dataset = train_dataset.map(parse_csv)      # parse each row
train_dataset = train_dataset.shuffle(buffer_size=1000)  # randomize
train_dataset = train_dataset.batch(32)

In [6]:
data, label = list(train_dataset)[0]

In [17]:
tf.erfc(model(data))

<tf.Tensor: id=246, shape=(32, 3), dtype=float32, numpy=
array([[1.9999787, 1.9538559, 1.9227166],
       [1.9997761, 1.9317652, 1.8262267],
       [1.9998621, 1.8904272, 1.8117819],
       [1.9999994, 1.9835274, 1.9038116],
       [1.9999663, 1.9940877, 1.3847582],
       [1.9995071, 1.9392292, 1.5699806],
       [1.9995978, 1.955271 , 1.7002175],
       [1.9994366, 1.975436 , 1.4686047],
       [1.9999927, 1.9510083, 1.9061899],
       [1.999969 , 1.9464321, 1.9528457],
       [1.9999958, 1.913749 , 1.9717603],
       [1.9999821, 1.9610052, 1.8669156],
       [1.9983909, 1.9742976, 1.462902 ],
       [1.9999709, 1.9975294, 1.2071185],
       [1.9997008, 1.9918481, 1.2521313],
       [1.9998415, 1.89848  , 1.8287635],
       [1.9999628, 1.9036245, 1.9186908],
       [1.9988068, 1.9684327, 1.5191865],
       [1.9996684, 1.9618807, 1.8072703],
       [1.9984097, 1.9679722, 1.4764254],
       [1.999956 , 1.9395852, 1.7995771],
       [1.9999458, 1.9555118, 1.9226637],
       [1.9999006, 

In [5]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.01)

train_loss_results = []
train_accuracy_results = []

num_epochs = 201

for epoch in range(num_epochs):
    epoch_loss_avg = tfe.metrics.Mean()
    epoch_accuracy = tfe.metrics.Accuracy()
    
    for x, y in train_dataset:
        grads = grad(model, x, y)
        optimizer.apply_gradients(zip(grads, model.variables), global_step=tf.train.get_or_create_global_step())
        epoch_loss_avg(loss(model, x, y))
        epoch_accuracy(tf.argmax(model(x), axis = 1, output_type = tf.int32), y)
        
    train_loss_results.append(epoch_loss_avg.result())
    train_accuracy_results.append(epoch_accuracy.result())
    
    if epoch % 50 == 0:
        print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch, epoch_loss_avg.result(), epoch_accuracy.result()))

Epoch 000: Loss: 1.253, Accuracy: 3.333%
Epoch 050: Loss: 0.418, Accuracy: 92.500%
Epoch 100: Loss: 0.221, Accuracy: 96.667%
Epoch 150: Loss: 0.153, Accuracy: 97.500%
Epoch 200: Loss: 0.120, Accuracy: 96.667%


In [6]:
test_dataset_url = "http://download.tensorflow.org/data/iris_test.csv"
test_dataset_fp = tf.keras.utils.get_file(fname=os.path.basename(test_dataset_url),origin=test_dataset_url)

test_dataset = tf.data.TextLineDataset(test_dataset_fp)

test_dataset = test_dataset.skip(1)             # skip the first header row
test_dataset = test_dataset.map(parse_csv)      # parse each row
test_dataset = test_dataset.shuffle(buffer_size=1000)  # randomize
test_dataset = test_dataset.batch(32)

In [7]:
test_accuracy = tfe.metrics.Accuracy()

for x,y in test_dataset:
    prediction = tf.argmax(model(x), axis = 1, output_type = tf.int32)
    test_accuracy(prediction, y)
    
print("Test set accuracy: {:.3%}".format(test_accuracy.result()))

Test set accuracy: 96.667%


In [8]:
class_ids = ["Iris setosa", "Iris versicolor", "Iris virginica"]

predict_dataset = tf.convert_to_tensor([
    [5.1, 3.3, 1.7, 0.5,],
    [5.9, 3.0, 4.2, 1.5,],
    [6.9, 3.1, 5.4, 2.1]
])

predictions = model(predict_dataset)

for i, logits in enumerate(predictions):
    class_idx = tf.argmax(logits).numpy()
    name = class_ids[class_idx]
    print("Example {} prediction: {}".format(i, name))

Example 0 prediction: Iris setosa
Example 1 prediction: Iris versicolor
Example 2 prediction: Iris virginica
